In [1]:
# Исследовать структуру html-файлов, чтобы произвести парсинг всех данных. 
# В каждом файле содержится информация об одном или нескольких объектах из случайной предметной области. 
# Перечень всех характеристик объекта может меняться (у отдельного объекта могут отсутствовать некоторые характеристики). 
# Полученные данные собрать и записать в json. 
# Выполните также ряд операций с данными:
# отсортируйте значения по одному из доступных полей
# выполните фильтрацию по другому полю (запишите результат отдельно)
# для одного выбранного числового поля посчитайте статистические характеристики (сумма, мин/макс, среднее и т.д.)
# для одного текстового поля посчитайте частоту меток.

In [4]:
from bs4 import BeautifulSoup
import re
import json
import math
import collections
import pandas as pd

In [5]:
def handle_file(file_name):
    items = list()
    with open(file_name, encoding = 'utf-8') as file:
        text = ""
        for row in file.readlines():
            text += row
        site = BeautifulSoup(text, 'html.parser')
        products = site.find_all('div', attrs={'class' : 'product-item'})
        #print(len(products))
        for product in products:
            item = dict()
            item['id'] = product.a['data-id']
            item['link'] = product.find_all('a')[1]['href']
            item['img'] = site.find_all('img')[0]['src']
            item['name'] = product.find_all('span')[0].get_text().strip()
            item['price'] = int(product.price.get_text().replace('₽', '').replace(' ', '').strip())
            item['bonus'] = int(product.strong.get_text().replace('+ начислим ', '').replace(' бонусов', '').strip())
            props = product.ul.find_all('li')
            for prop in props:
                item[prop['type']] = prop.get_text().strip()    
            items.append(item)
    return(items)       

In [8]:
items = []
for i in range(1,67):
  file_name = f'./2/{i}.html'
  items += handle_file(file_name)
print(items[1:5])    

[{'id': '77805', 'link': '/product/77805', 'img': '/upload/607974.png', 'name': '5.3" Kingston 32GB', 'price': 42425, 'bonus': 424, 'sim': '2 SIM', 'acc': '3098 мА * ч'}, {'id': '36288', 'link': '/product/36288', 'img': '/upload/607974.png', 'name': '6.4" Lenovo 208GB', 'price': 239901, 'bonus': 2399, 'processor': '8x2.1 ГГц', 'ram': '7 GB', 'matrix': 'IPS'}, {'id': '31232', 'link': '/product/31232', 'img': '/upload/607974.png', 'name': '7.5" Toshiba 96GB', 'price': 203430, 'bonus': 2034, 'processor': '4x4.1 ГГц', 'ram': '10 GB', 'matrix': 'IPS'}, {'id': '31245', 'link': '/product/31245', 'img': '/upload/607974.png', 'name': '7.1" OPPO 192GB', 'price': 90997, 'bonus': 909, 'ram': '15 GB', 'camera': '71 MP'}]


In [7]:
with open('result_all_17_2.json', 'w', encoding = 'utf-8') as f:
    f.write(json.dumps(items, ensure_ascii=False))

In [9]:
# Отсортируeм значения по полю ['bonus'] в убывающем порядке:
items = sorted(items, key = lambda x: x['bonus'], reverse = True)
print(items[1:5])

[{'id': '78327', 'link': '/product/78327', 'img': '/upload/307857.jpeg', 'name': '6.2" Sony 48GB', 'price': 499453, 'bonus': 4994, 'matrix': 'IPS', 'camera': '112 MP'}, {'id': '60395', 'link': '/product/60395', 'img': '/upload/64643.jpg', 'name': '6.4" IBM 16GB', 'price': 499434, 'bonus': 4994, 'processor': '4x4.0 ГГц', 'resolution': '1600x1440', 'acc': '3258 мА * ч'}, {'id': '88290', 'link': '/product/88290', 'img': '/upload/40787.svg', 'name': '5.0" Plextor 160GB', 'price': 499251, 'bonus': 4992, 'sim': '1 SIM', 'matrix': 'AMOLED', 'resolution': '1920x1200', 'acc': '3269 мА * ч'}, {'id': '48290', 'link': '/product/48290', 'img': '/upload/799496.png', 'name': '5.1" Microsoft 144GB', 'price': 499054, 'bonus': 4990, 'processor': '6x3.3 ГГц', 'ram': '3 GB'}]


In [14]:
# Выполним фильтрацию по полю ['price']:
filtered_items = []
for smartphone in items:
    if smartphone['price'] > 50000:
        filtered_items.append(smartphone)
print(filtered_items[1:5])        

[{'id': '78327', 'link': '/product/78327', 'img': '/upload/307857.jpeg', 'name': '6.2" Sony 48GB', 'price': 499453, 'bonus': 4994, 'matrix': 'IPS', 'camera': '112 MP'}, {'id': '60395', 'link': '/product/60395', 'img': '/upload/64643.jpg', 'name': '6.4" IBM 16GB', 'price': 499434, 'bonus': 4994, 'processor': '4x4.0 ГГц', 'resolution': '1600x1440', 'acc': '3258 мА * ч'}, {'id': '88290', 'link': '/product/88290', 'img': '/upload/40787.svg', 'name': '5.0" Plextor 160GB', 'price': 499251, 'bonus': 4992, 'sim': '1 SIM', 'matrix': 'AMOLED', 'resolution': '1920x1200', 'acc': '3269 мА * ч'}, {'id': '48290', 'link': '/product/48290', 'img': '/upload/799496.png', 'name': '5.1" Microsoft 144GB', 'price': 499054, 'bonus': 4990, 'processor': '6x3.3 ГГц', 'ram': '3 GB'}]


In [15]:
with open("result_filtered_17_2.json", "w", encoding = 'utf-8') as file:
    file.write(json.dumps(filtered_items, ensure_ascii=False))

In [16]:
# для числового поля ['bonus'] посчитаем статистические характеристики (сумма, мин/макс, среднее и т.д.)
stats_items = []
df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)
stats = df['bonus'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
stats_items.append(stats)
print(stats_items)

[{'max': 4995.0, 'min': 101.0, 'mean': 2560.7774287095217, 'median': 2545.5, 'std': 1394.0501260467024}]


In [21]:
# для текстового поля ['name'] посчитайте частоту меток
text_items = []
text = [item['name'] for item in items]
f1 = collections.Counter(text)
text_items.append(f1)
print(text_items)

[Counter({'6.3" Acer 192GB': 4, '5.9" SanDisk 112GB': 4, '6.8" OCZ 48GB': 3, '7.5" VIVO 256GB': 3, '5.8" ADATA 240GB': 3, '7.4" VIVO 96GB': 3, '5.2" Huawei P 96GB': 3, '6.3" Lenovo 16GB': 3, '7.3" OPPO 32GB': 3, '5.9" Sony 144GB': 3, '6.2" Dell 208GB': 3, '7.5" Acer 192GB': 3, '7.4" Broadcom 224GB': 3, '6.6" Dell 176GB': 3, '6.6" Toshiba 96GB': 3, '5.1" Samsung 128GB': 3, '6.1" Toshiba 32GB': 3, '7.2" Lenovo 128GB': 3, '6.1" LG 176GB': 3, '5.1" Xiaomi Mi 16GB': 3, '7.2" Lenovo 176GB': 3, '6.2" Lenovo 96GB': 3, '6.6" VIVO 96GB': 3, '6.9" Microsoft 208GB': 3, '7.0" Dell 208GB': 3, '5.7" Dell 64GB': 3, '6.1" ADATA 16GB': 3, '6.8" SanDisk 224GB': 3, '5.1" SanDisk 96GB': 3, '5.7" OCZ 48GB': 3, '5.1" Lenovo 32GB': 3, '6.3" Intel 64GB': 3, '6.1" OPPO 144GB': 3, '5.8" Dell 64GB': 3, '7.5" Dell 32GB': 2, '6.6" Google Pixel 192GB': 2, '6.5" Transcend 80GB': 2, '5.8" Samsung Note 208GB': 2, '5.0" Transcend 240GB': 2, '5.2" Dell 48GB': 2, '6.9" Western Digital 240GB': 2, '5.7" Apple 80GB': 2, '5.0